<a href="https://colab.research.google.com/github/ckkhandare/svsvddvd/blob/main/Untitled12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

trial

vd fd 

zvsvsd